# DATASETS

In [63]:
import bootcampviztools as bt
import hashlib
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

#### DEBILIDADES Y FORTALEZAS 

DC

In [64]:
# Cargar data DC
dc_df = pd.read_csv('./data/DC_extended_universe.csv')

In [65]:
# Creo nuevo data, solo con nombre
data_DC_nombre = dc_df[['nombre']]
data_DC_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  73 non-null     object
dtypes: object(1)
memory usage: 716.0+ bytes


In [66]:
# Guardar data en .csv
data_DC_nombre.to_csv('./data/DC_nombres.csv', index = False)

In [67]:
# Fortalezas y debilidades basadas en cómics y películas
strengths_weaknesses_DC = {
    "Aquaman": {
        "Fortalezas": ["Control sobre animales marinos", "Fuerza sobrehumana", "Durabilidad", "Habilidades de combate", "Comunicación telepática con la vida marina"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente", "Vulnerabilidad a la deshidratación"]
    },
    "Ares": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades mágicas", "Control sobre el caos y la guerra", "Inmortalidad"],
        "Debilidades": ["Dependencia del caos y conflicto para mantener su poder"]
    },
    "Atom Smasher": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada con el tamaño", "Resistencia"],
        "Debilidades": ["Difícil de controlar su tamaño", "Problemas con la coordinación al cambiar de tamaño"]
    },
    "Batman": {
        "Fortalezas": ["Inteligencia de nivel genio", "Maestro en artes marciales", "Detección avanzada", "Equipamiento y gadgets", "Habilidades de detective"],
        "Debilidades": ["Humano sin poderes sobrenaturales", "Vulnerabilidad física"]
    },
    "Black Adam": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Magia", "Vuelo", "Resistencia a la magia"],
        "Debilidades": ["Vulnerable a la magia poderosa", "Arrogancia"]
    },
    "Black Canary": {
        "Fortalezas": ["Grito sónico", "Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales fuera de su grito"]
    },
    "Black Manta": {
        "Fortalezas": ["Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Black Mask": {
        "Fortalezas": ["Estratega criminal", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Bloodsport": {
        "Fortalezas": ["Experto en armas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Captain Boomerang": {
        "Fortalezas": ["Habilidad con boomerangs", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Cassandra Cain": {
        "Fortalezas": ["Habilidades de combate", "Experta en artes marciales"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Cheetah": {
        "Fortalezas": ["Fuerza y agilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de la transformación"]
    },
    "Cyborg": {
        "Fortalezas": ["Tecnología avanzada", "Fuerza sobrehumana", "Habilidades de hackeo"],
        "Debilidades": ["Dependencia de su tecnología"]
    },
    "Cyclone": {
        "Fortalezas": ["Control del viento", "Vuelo"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Dark Flash": {
        "Fortalezas": ["Velocidad sobrehumana", "Reflejos rápidos"],
        "Debilidades": ["Dependencia de la Speed Force"]
    },
    "Darkseid": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades mágicas", "Control sobre el caos y la guerra", "Inmortalidad"],
        "Debilidades": ["Dependencia del caos y conflicto para mantener su poder"]
    },
    "Deadshot": {
        "Fortalezas": ["Experto en armas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Doctor Fate": {
        "Fortalezas": ["Habilidades mágicas", "Inmortalidad", "Vuelo"],
        "Debilidades": ["Dependencia del casco"]
    },
    "Doctor Sivana": {
        "Fortalezas": ["Intelecto de nivel genio", "Habilidades científicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Doomsday": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la magia"]
    },
    "El Diablo": {
        "Fortalezas": ["Control del fuego", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales fuera de su control de fuego"]
    },
    "Enchantress": {
        "Fortalezas": ["Habilidades mágicas", "Control mental"],
        "Debilidades": ["Dependencia de su magia"]
    },
    "Envy": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Dependencia de la envidia"]
    },
    "Faora": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la kryptonita"]
    },
    "Freddy Freeman": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo"],
        "Debilidades": ["Dependencia de la magia de Shazam"]
    },
    "General Zod": {
        "Fortalezas": ["Fuerza y durabilidad sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la kryptonita"]
    },
    "Gluttony": {
        "Fortalezas": ["Fuerza sobrehumana"],
        "Debilidades": ["Dependencia de la gula"]
    },
    "Green Lantern": {
        "Fortalezas": ["Creación de construcciones de luz sólida", "Vuelo", "Viaje interestelar", "Resistencia"],
        "Debilidades": ["Vulnerabilidad al color amarillo (dependiendo de la versión)", "Dependencia del anillo y su carga"]
    },
    "Harley Quinn": {
        "Fortalezas": ["Habilidades de combate", "Inteligencia"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Hawkman": {
        "Fortalezas": ["Vuelo", "Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de su armadura"]
    },
    "Hespera": {
        "Fortalezas": ["Habilidades mágicas"],
        "Debilidades": ["Dependencia de su magia"]
    },
    "Huntress": {
        "Fortalezas": ["Habilidades de combate", "Experta en armas"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Incubus": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Dependencia de la lujuria"]
    },
    "Joker": {
        "Fortalezas": ["Inteligencia", "Habilidades estratégicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Jor-El": {
        "Fortalezas": ["Inteligencia"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Judomaster": {
        "Fortalezas": ["Habilidades de combate", "Experta en artes marciales"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Karathen": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Katana": {
        "Fortalezas": ["Habilidades de combate", "Experta en armas"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Killer Croc": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "King Nereus": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "King Shark": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad física"]
    },
        "Lex Luthor": {
        "Fortalezas": ["Inteligencia", "Habilidades estratégicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Lust": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Dependencia de la lujuria"]
    },
    "Mera": {
        "Fortalezas": ["Control sobre el agua", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "Mister Zsasz": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Nam-Ek": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Vulnerable a la kryptonita"]
    },
    "Nuidis Vulko": {
        "Fortalezas": ["Inteligencia", "Habilidades estratégicas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Ocean Master": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "Parademon": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de Darkseid"]
    },
    "Peacemaker": {
        "Fortalezas": ["Habilidades de combate", "Experto en armas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Penthesileia": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Philippus": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Polka-Dot Man": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Queen Atlanna": {
        "Fortalezas": ["Control sobre el agua", "Habilidades de combate"],
        "Debilidades": ["Necesidad de estar en contacto con el agua regularmente"]
    },
    "Queen Hippolyta": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Inmortalidad"],
        "Debilidades": ["Dependencia de la inmortalidad"]
    },
    "Ratcatcher": {
        "Fortalezas": ["Control sobre ratas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Rick Flag": {
        "Fortalezas": ["Habilidades de combate", "Liderazgo"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Sabbac": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades mágicas", "Vuelo"],
        "Debilidades": ["Dependencia de la magia"]
    },
    "Savant": {
        "Fortalezas": ["Intelecto de nivel genio", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Shazam": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo", "Magia"],
        "Debilidades": ["Dependencia de la magia de Shazam"]
    },
    "Slipknot": {
        "Fortalezas": ["Habilidad con cuerdas", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Sol Soria": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Starro": {
        "Fortalezas": ["Control mental", "Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad a la separación física"]
    },
    "Steppenwolf": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Inmortalidad"],
        "Debilidades": ["Dependencia de Darkseid"]
    },
    "Supergirl": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo", "Visión de rayos X", "Invulnerabilidad", "Visión calorífica", "Aliento helado"],
        "Debilidades": ["Kryptonita", "Magia"]
    },
    "Superman": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad", "Vuelo", "Visión de rayos X", "Invulnerabilidad", "Visión calorífica", "Aliento helado"],
        "Debilidades": ["Kryptonita", "Magia"]
    },
    "T.D.K.": {
        "Fortalezas": ["Desprendimiento de extremidades", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales fuera de su habilidad"]
    },
    "The Flash": {
        "Fortalezas": ["Velocidad sobrehumana", "Reflejos rápidos", "Intangibilidad", "Viaje en el tiempo", "Curación acelerada"],
        "Debilidades": ["Fuerzas de fricción extremas", "Dependencia de la Speed Force"]
    },
    "Vigilante": {
        "Fortalezas": ["Habilidades de combate", "Experto en armas"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "White Dragon": {
        "Fortalezas": ["Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Wonder Woman (Armor of Asteria)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Vuelo", "Agilidad", "Resistencia"],
        "Debilidades": ["Dependencia de la verdad"]
    },
    "Wrath": {
        "Fortalezas": ["Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Zeus": {
        "Fortalezas": ["Habilidades mágicas", "Inmortalidad", "Control del rayo"],
        "Debilidades": ["Dependencia de su magia"]
    }
}


In [68]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_DC(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_DC:
        return pd.Series([
            strengths_weaknesses_DC[hero_name]['Fortalezas'],
            strengths_weaknesses_DC[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [69]:
dc_df[['Fortalezas', 'Debilidades']] = dc_df.apply(assign_strenghs_weaknesses_DC, axis = 1)

In [70]:
dc_df.info()
dc_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  61 non-null     object
 1   nombre           73 non-null     object
 2   universo         73 non-null     object
 3   url              73 non-null     object
 4   Intelligence     73 non-null     int64 
 5   Strength         73 non-null     int64 
 6   Speed            73 non-null     int64 
 7   Durability       73 non-null     int64 
 8   Power            73 non-null     int64 
 9   Combat           73 non-null     int64 
 10  Tier             73 non-null     int64 
 11  Tier2            9 non-null      object
 12  superpoderes     73 non-null     object
 13  heroe            73 non-null     object
 14  Fortalezas       73 non-null     object
 15  Debilidades      73 non-null     object
dtypes: int64(7), object(9)
memory usage: 9.3+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,"['Animal Control', 'Durability', 'Stamina', 'S...",Aquaman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...
1,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,"['Accelerated Healing', 'Chain Manipulation', ...",Ares,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
2,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,"['Jaw Strength', 'Large Size', 'Size Changing'...",Atom Smasher,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,"['Bloodlust', 'Dexterity', 'Disguise Mastery',...",Batman,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,"['Afterimage Creation', 'Agility', 'Cold Resis...",Black Adam,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"


THE BOYS 

In [71]:
theboys_df = pd.read_csv('./data/The_boys.csv')

In [72]:
# Creo nuevo data, solo con nombre
data_TB_nombre = theboys_df[['nombre']]
data_TB_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  35 non-null     object
dtypes: object(1)
memory usage: 412.0+ bytes


In [73]:
# Guardar data en .csv
data_TB_nombre.to_csv('./data/TB_nombres.csv', index = False)

In [74]:
# Lista de fortalezas y debilidades basadas en la serie "The Boys"
strengths_weaknesses_TB = {
    "A-Train": {
        "Fortalezas": ["Velocidad sobrehumana"],
        "Debilidades": ["Problemas cardíacos", "Dependencia de Compuesto V"]
    },
    "Billy Butcher": {
        "Fortalezas": ["Habilidades de combate", "Liderazgo"],
        "Debilidades": ["Humano sin poderes sobrenaturales", "Impulsividad"]
    },
    "Billy Butcher (V24)": {
        "Fortalezas": ["Fuerza sobrehumana (temporal)", "Habilidades de combate"],
        "Debilidades": ["Efectos secundarios de V24", "Tiempo limitado de poderes"]
    },
    "Billy Butcher (tumor)": {
        "Fortalezas": ["Determinación", "Habilidades de combate"],
        "Debilidades": ["Enfermedad terminal", "Humano sin poderes sobrenaturales"]
    },
    "Black Noir (replacement)": {
        "Fortalezas": ["Habilidades de combate", "Sigilo"],
        "Debilidades": ["Desconocido"]
    },
    "Black Noir": {
        "Fortalezas": ["Habilidades de combate", "Resistencia sobrehumana"],
        "Debilidades": ["Vulnerabilidad a heridas graves", "Trauma psicológico"]
    },
    "Cate Dunlap": {
        "Fortalezas": ["Habilidades desconocidas"],
        "Debilidades": ["Desconocido"]
    },
    "Crimson Countess": {
        "Fortalezas": ["Manipulación de energía", "Vuelo"],
        "Debilidades": ["Dependencia emocional", "Humano sin invulnerabilidad"]
    },
    "Firecracker": {
        "Fortalezas": ["Piroquinesis"],
        "Debilidades": ["Desconocido"]
    },
    "Golden Boy": {
        "Fortalezas": ["Fuerza sobrehumana", "Piroquinesis"],
        "Debilidades": ["Arrogancia", "Impulsividad"]
    },
    "Homelander": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Visión calorífica", "Invulnerabilidad"],
        "Debilidades": ["Inestabilidad mental", "Narcisismo"]
    },
    "Hughie Campbell (V24)": {
        "Fortalezas": ["Fuerza sobrehumana (temporal)", "Teleportación (temporal)"],
        "Debilidades": ["Efectos secundarios de V24", "Tiempo limitado de poderes"]
    },
    "Lamplighter": {
        "Fortalezas": ["Manipulación del fuego"],
        "Debilidades": ["Dependencia emocional", "Trauma psicológico"]
    },
    "Little Cricket": {
        "Fortalezas": ["Cambio de tamaño"],
        "Debilidades": ["Vulnerabilidad física cuando es pequeña"]
    },
    "Marie Moreau": {
        "Fortalezas": ["Control de la sangre"],
        "Debilidades": ["Desconocido"]
    },
    "Mesmer": {
        "Fortalezas": ["Lectura de mentes"],
        "Debilidades": ["Vulnerabilidad física", "Humano sin poderes de combate"]
    },
    "Mother's Milk": {
        "Fortalezas": ["Habilidades de combate", "Estrategia"],
        "Debilidades": ["Humano sin poderes sobrenaturales", "Trauma emocional"]
    },
    "Naqib": {
        "Fortalezas": ["Explosión de energía"],
        "Debilidades": ["Desconocido"]
    },
    "Queen Maeve": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia emocional", "Humano sin invulnerabilidad total"]
    },
    "Ryan Butcher": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Visión calorífica"],
        "Debilidades": ["Inexperiencia", "Emocionalmente inestable"]
    },
    "Sam Riordan": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Shockwave": {
        "Fortalezas": ["Velocidad sobrehumana"],
        "Debilidades": ["Competencia con A-Train", "Dependencia de Compuesto V"]
    },
    "Sister Sage": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Soldier Boy": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Resistencia"],
        "Debilidades": ["Trauma psicológico", "Narcisismo"]
    },
    "Splinter": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Starlight": {
        "Fortalezas": ["Manipulación de la luz", "Vuelo"],
        "Debilidades": ["Dependencia emocional", "Humano sin invulnerabilidad total"]
    },
    "Stormfront": {
        "Fortalezas": ["Manipulación de electricidad", "Vuelo", "Fuerza sobrehumana"],
        "Debilidades": ["Ideología extremista", "Vulnerabilidad emocional"]
    },
    "Supe Terrorist": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Tek Knight": {
        "Fortalezas": ["Tecnología avanzada", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "The Deep": {
        "Fortalezas": ["Comunicación con la vida marina", "Habilidades de combate"],
        "Debilidades": ["Inestabilidad emocional", "Humano sin invulnerabilidad"]
    },
    "The Female": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Regeneración"],
        "Debilidades": ["Trauma emocional"]
    },
    "Translucent": {
        "Fortalezas": ["Invisibilidad", "Piel a prueba de balas"],
        "Debilidades": ["Vulnerabilidad a explosivos internos"]
    },
    "Victoria Neuman": {
        "Fortalezas": ["Explosión de cabezas"],
        "Debilidades": ["Secretismo", "Humano sin invulnerabilidad"]
    },
    "Webweaver": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Zoe Neuman": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    }
}


In [75]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_Theboys(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_TB:
        return pd.Series([
            strengths_weaknesses_TB[hero_name]['Fortalezas'],
            strengths_weaknesses_TB[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [76]:
theboys_df[['Fortalezas', 'Debilidades']] = theboys_df.apply(assign_strenghs_weaknesses_Theboys, axis = 1)

In [77]:
theboys_df.info()
theboys_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  17 non-null     object
 1   nombre           35 non-null     object
 2   universo         35 non-null     object
 3   url              35 non-null     object
 4   Intelligence     35 non-null     int64 
 5   Strength         35 non-null     int64 
 6   Speed            35 non-null     int64 
 7   Durability       35 non-null     int64 
 8   Power            35 non-null     int64 
 9   Combat           35 non-null     int64 
 10  Tier             35 non-null     int64 
 11  Tier2            22 non-null     object
 12  superpoderes     35 non-null     object
 13  heroe            35 non-null     object
 14  Fortalezas       35 non-null     object
 15  Debilidades      35 non-null     object
dtypes: int64(7), object(9)
memory usage: 4.5+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,"Reginald ""Reggie"" C. Franklin",A-Train,The Boys (TV Series 2019),https://www.superherodb.com/a-train/10-21495/,60,25,25,15,35,30,1,1+,"['Agility', 'Enhanced Condition', 'Enhanced He...",A-Train,[Velocidad sobrehumana],"[Problemas cardíacos, Dependencia de Compuesto V]"
1,William Butcher,Billy Butcher,The Boys (TV Series 2019),https://www.superherodb.com/billy-butcher/10-2...,65,5,7,8,10,60,1,0+,"['Agility', 'Dexterity', 'Marksmanship', 'Mast...",Billy Butcher,"[Habilidades de combate, Liderazgo]","[Humano sin poderes sobrenaturales, Impulsividad]"
2,William Butcher,Billy Butcher (V24),The Boys (TV Series 2019),https://www.superherodb.com/billy-butcher-v24/...,65,25,8,20,50,60,1,NaN,"['Heat Generation', 'Stamina', 'Toxin and Dise...",Billy Butcher (V24),"[Fuerza sobrehumana (temporal), Habilidades de...","[Efectos secundarios de V24, Tiempo limitado d..."
3,William Butcher,Billy Butchter (tumor),The Boys (TV Series 2019),https://www.superherodb.com/billy-butchter-tum...,65,25,14,20,35,65,1,1,"['Instinctive Reaction', 'Rage Power', 'Super ...",Billy Butcher (Tumor),[Desconocido],[Desconocido]
4,NaN,Black Noir (replacement),The Boys (TV Series 2019),https://www.superherodb.com/black-noir-replace...,50,25,25,15,35,55,1,1,"['Insanity', 'Stamina', 'Super Speed', 'Super ...",Black Noir (Replacement),"[Habilidades de combate, Sigilo]",[Desconocido]


MCU

In [78]:
mcu_df = pd.read_csv('./data/MCU.csv')

In [79]:
# Creo nuevo data, solo con nombre
data_MCU_nombre = mcu_df[['nombre']]
data_MCU_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  176 non-null    object
dtypes: object(1)
memory usage: 1.5+ KB


In [80]:
# Guardar data en .csv
data_MCU_nombre.to_csv('./data/MCU_nombres.csv', index = False)

In [81]:
# Fortalezas y debilidades basadas en personajes del MCU
strengths_weaknesses_MCU = {
    "Agatha Harkness": {
        "Fortalezas": ["Magia oscura", "Manipulación mágica"],
        "Debilidades": ["Necesita absorber la magia de otros para mantener su poder"]
    },
    "Abomination": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Vulnerable a ataques mentales"]
    },
    "Ajak": {
        "Fortalezas": ["Curación", "Fuerza y durabilidad sobrehumanas"],
        "Debilidades": ["Vulnerable a traiciones de otros Eternos"]
    },
    "Ancient One (time stone)": {
        "Fortalezas": ["Manipulación del tiempo", "Magia avanzada"],
        "Debilidades": ["Dependencia de la Piedra del Tiempo"]
    },
    "Ancient One": {
        "Fortalezas": ["Magia avanzada", "Longevidad"],
        "Debilidades": ["Vulnerabilidad física sin la Piedra del Tiempo"]
    },
    "Ant-Man": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada al reducirse"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Arishem The Judge": {
        "Fortalezas": ["Poder cósmico", "Control sobre los Celestiales"],
        "Debilidades": ["Desconocido"]
    },
    "Ayo": {
        "Fortalezas": ["Habilidades de combate", "Lealtad"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Baby Groot": {
        "Fortalezas": ["Regeneración", "Habilidades de combate"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Baron Mordo": {
        "Fortalezas": ["Magia avanzada", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Desconfianza"]
    },
    "Baron Zemo": {
        "Fortalezas": ["Inteligencia estratégica", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Black Bolt (Earth 838)": {
        "Fortalezas": ["Voz destructiva", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad al control mental"]
    },
    "Black Bolt": {
        "Fortalezas": ["Voz destructiva", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad al control mental"]
    },
    "Black Panther": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Traje avanzado"],
        "Debilidades": ["Dependencia del traje y hierba en forma de corazón"]
    },
    "Black Widow": {
        "Fortalezas": ["Habilidades de combate", "Espionaje"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Bucky Barnes": {
        "Fortalezas": ["Fuerza sobrehumana", "Brazo biónico", "Habilidades de combate"],
        "Debilidades": ["Trauma psicológico", "Programación mental"]
    },
    "Bullseye": {
        "Fortalezas": ["Precisión letal", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Captain America": {
        "Fortalezas": ["Fuerza sobrehumana", "Liderazgo", "Habilidades de combate"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Captain America (Worthy)": {
        "Fortalezas": ["Fuerza sobrehumana", "Liderazgo", "Habilidades de combate", "Puede manejar el Mjolnir"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Captain Carter (Earth 838)": {
        "Fortalezas": ["Fuerza sobrehumana", "Liderazgo", "Habilidades de combate"],
        "Debilidades": ["Humana con límites físicos"]
    },
    "Captain Marvel": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Energía fotónica"],
        "Debilidades": ["Arrogancia", "Dependencia de sus poderes"]
    },
    "Chitauri Footsoldier": {
        "Fortalezas": ["Fuerza sobrehumana", "Armas avanzadas"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Corvus Glaive": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de su lanza"]
    },
    "Crossbones": {
        "Fortalezas": ["Fuerza mejorada", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Cull Obsidian": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Dar-Benn": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Daredevil": {
        "Fortalezas": ["Sentidos mejorados", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Deathlok": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de su tecnología"]
    },
    "Destroyer": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia", "Armamento avanzado"],
        "Debilidades": ["Controlado externamente"]
    },
    "Doctor Strange": {
        "Fortalezas": ["Magia avanzada", "Manipulación del tiempo (con la Piedra del Tiempo)"],
        "Debilidades": ["Dependencia de artefactos mágicos"]
    },
    "Dormammu": {
        "Fortalezas": ["Poder cósmico", "Inmortalidad"],
        "Debilidades": ["Vulnerabilidad a la manipulación del tiempo"]
    },
    "Drax The Destroyer": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Ebony Maw": {
        "Fortalezas": ["Telequinesis", "Inteligencia"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Echo": {
        "Fortalezas": ["Habilidades de combate", "Mimetismo"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Ego": {
        "Fortalezas": ["Poder cósmico", "Manipulación de materia"],
        "Debilidades": ["Dependencia de su núcleo"]
    },
    "Elektra": {
        "Fortalezas": ["Habilidades de combate", "Sigilo"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Eternity": {
        "Fortalezas": ["Poder cósmico", "Omnisciencia"],
        "Debilidades": ["Desconocido"]
    },
    "Falcon": {
        "Fortalezas": ["Habilidades de combate", "Vuelo con traje alado"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Fandral": {
        "Fortalezas": ["Habilidades de combate", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Fat Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Resistencia"],
        "Debilidades": ["Estado emocional inestable"]
    },
    "Fenris": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Vulnerabilidad a armas asgardianas"]
    },
    "Frigga": {
        "Fortalezas": ["Magia avanzada", "Habilidades de combate"],
        "Debilidades": ["Humana con límites físicos"]
    },
    "G'iah": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Gamora": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Gargantos": {
        "Fortalezas": ["Fuerza sobrehumana", "Múltiples tentáculos"],
        "Debilidades": ["Desconocido"]
    },
    "Ghost": {
        "Fortalezas": ["Intangibilidad", "Invisibilidad"],
        "Debilidades": ["Dependencia de su traje"]
    },
    "Ghost Rider": {
        "Fortalezas": ["Piroquinesis", "Inmortalidad"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "Giant-Man": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Gilgamesh": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Gorr The God Butcher": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de la Necrosword"]
    },
    "Gravik": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Groot": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Hawkeye": {
        "Fortalezas": ["Precisión", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Heimdall": {
        "Fortalezas": ["Sentidos mejorados", "Fuerza sobrehumana"],
        "Debilidades": ["Desconocido"]
    },
    "Hela": {
        "Fortalezas": ["Fuerza sobrehumana", "Control sobre armas"],
        "Debilidades": ["Dependencia de Asgard"]
    },
    "High Evolutionary": {
        "Fortalezas": ["Inteligencia avanzada", "Manipulación genética"],
        "Debilidades": ["Arrogancia"]
    },
    "Hulk (Stark Gauntlet)": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Efectos secundarios del Gauntlet"]
    },
    "Hulk": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Falta de control en su forma de Hulk"]
    },
    "Ikaris": {
        "Fortalezas": ["Vuelo", "Fuerza sobrehumana", "Proyección de energía"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "Iron Fist": {
        "Fortalezas": ["Puño de hierro", "Habilidades de combate"],
        "Debilidades": ["Dependencia de Chi"]
    },
    "Iron Man": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Iron Man (Mk 1)": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Iron Man (Stark Gauntlet)": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Efectos secundarios del Gauntlet"]
    },
    "Iron Man (Hulkbuster)": {
        "Fortalezas": ["Fuerza incrementada", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Iron Monger": {
        "Fortalezas": ["Fuerza incrementada", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Iron Spider": {
        "Fortalezas": ["Fuerza sobrehumana", "Tecnología avanzada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Ironheart": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Jemiah The Analyzer": {
        "Fortalezas": ["Poder cósmico", "Inteligencia avanzada"],
        "Debilidades": ["Desconocido"]
    },
    "Jessica Jones": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Trauma emocional"]
    },
    "Kaecilius": {
        "Fortalezas": ["Magia avanzada", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Kang the Conqueror": {
        "Fortalezas": ["Manipulación del tiempo", "Inteligencia avanzada"],
        "Debilidades": ["Dependencia de la tecnología"]
    },
    "Killian": {
        "Fortalezas": ["Poder de regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Inestabilidad emocional"]
    },
    "Killmonger": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Kingo": {
        "Fortalezas": ["Proyección de energía", "Inmortalidad"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Kingpin": {
        "Fortalezas": ["Fuerza incrementada", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Korath The Pursuer": {
        "Fortalezas": ["Fuerza sobrehumana", "Tecnología avanzada"],
        "Debilidades": ["Desconocido"]
    },
    "Korg": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Vulnerabilidad a explosivos"]
    },
    "Kro": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Desconocido"]
    },
    "Kurse": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Desconocido"]
    },
    "Kid Loki": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Lady Loki": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Leo Fitz": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Leviathan": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Desconocido"]
    },
    "Loki": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (Mind Stone)": {
        "Fortalezas": ["Magia", "Control mental"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (Keeper Of The Multiverse)": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (Variant L1130)": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Loki (King of Asgard)": {
        "Fortalezas": ["Magia", "Inteligencia"],
        "Debilidades": ["Arrogancia"]
    },
    "Luke Cage": {
        "Fortalezas": ["Fuerza sobrehumana", "Piel a prueba de balas"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "M'baku": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Makkari": {
        "Fortalezas": ["Velocidad sobrehumana", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Malekith (Reality Stone)": {
        "Fortalezas": ["Manipulación de la realidad", "Fuerza sobrehumana"],
        "Debilidades": ["Dependencia de la Piedra de la Realidad"]
    },
    "Malekith": {
        "Fortalezas": ["Fuerza sobrehumana", "Manipulación de la oscuridad"],
        "Debilidades": ["Desconocido"]
    },
    "Mandarin": {
        "Fortalezas": ["Habilidades de combate", "Anillos mágicos"],
        "Debilidades": ["Dependencia de los anillos"]
    },
    "Mantis": {
        "Fortalezas": ["Empatía", "Manipulación emocional"],
        "Debilidades": ["Físicamente vulnerable"]
    },
    "Mighty Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Cáncer debilitante"]
    },
    "MODOK": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Físicamente limitado"]
    },
    "Moon Knight": {
        "Fortalezas": ["Habilidades de combate", "Resistencia incrementada"],
        "Debilidades": ["Trastorno de identidad disociativo"]
    },
    "Mr. Fantastic (Earth 838)": {
        "Fortalezas": ["Elasticidad", "Inteligencia"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Mr. Knight": {
        "Fortalezas": ["Habilidades de combate", "Resistencia incrementada"],
        "Debilidades": ["Trastorno de identidad disociativo"]
    },
    "Ms. America": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Viaje interdimensional"],
        "Debilidades": ["Inexperiencia"]
    },
    "Ms. Marvel": {
        "Fortalezas": ["Elasticidad", "Manipulación de energía"],
        "Debilidades": ["Inexperiencia"]
    },
    "Mysterio": {
        "Fortalezas": ["Ilusiones avanzadas", "Inteligencia"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Namor": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Control del agua"],
        "Debilidades": ["Necesita agua para mantener su fuerza"]
    },
    "Nebula": {
        "Fortalezas": ["Cibernética avanzada", "Habilidades de combate"],
        "Debilidades": ["Trauma emocional"]
    },
    "Nick Fury": {
        "Fortalezas": ["Estrategia", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Nomad": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Odin": {
        "Fortalezas": ["Magia avanzada", "Fuerza sobrehumana", "Longevidad"],
        "Debilidades": ["Depende del Odinsleep para rejuvenecerse"]
    },
    "Okoye": {
        "Fortalezas": ["Habilidades de combate", "Lealtad"],
        "Debilidades": ["Humana sin poderes sobrenaturales"]
    },
    "Outrider": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Inteligencia limitada"]
    },
    "Pepper Potts (Extremis)": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Inestabilidad de Extremis"]
    },
    "Phastos": {
        "Fortalezas": ["Tecnología avanzada", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Photon": {
        "Fortalezas": ["Manipulación de energía", "Vuelo"],
        "Debilidades": ["Desconocido"]
    },
    "Proxima Midnight": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Dependencia de su lanza"]
    },
    "Quake": {
        "Fortalezas": ["Manipulación de vibraciones"],
        "Debilidades": ["Vulnerabilidad física"]
    },
    "Quicksilver": {
        "Fortalezas": ["Velocidad sobrehumana"],
        "Debilidades": ["Impulsividad"]
    },
    "Red Guardian": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Humano con límites físicos"]
    },
    "Red Skull (Stonekeeper)": {
        "Fortalezas": ["Inmortalidad", "Conocimiento de las gemas del infinito"],
        "Debilidades": ["Encadenado a Vormir"]
    },
    "Red Skull": {
        "Fortalezas": ["Intelecto avanzado", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Rescue": {
        "Fortalezas": ["Tecnología avanzada", "Vuelo"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Rocket Raccoon": {
        "Fortalezas": ["Inteligencia", "Habilidades de combate"],
        "Debilidades": ["Tamaño pequeño"]
    },
    "Ronan The Accuser (Power Stone)": {
        "Fortalezas": ["Fuerza sobrehumana", "Poder de la gema del poder"],
        "Debilidades": ["Dependencia de la gema del poder"]
    },
    "Ronan The Accuser": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia"]
    },
    "Ronin": {
        "Fortalezas": ["Habilidades de combate", "Sigilo"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Scarlet Witch": {
        "Fortalezas": ["Magia del caos", "Manipulación de la realidad"],
        "Debilidades": ["Inestabilidad emocional"]
    },
    "Sersi (uni-mind)": {
        "Fortalezas": ["Manipulación de materia", "Inmortalidad"],
        "Debilidades": ["Dependencia del Uni-Mind"]
    },
    "Sersi": {
        "Fortalezas": ["Manipulación de materia", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Shang-Chi": {
        "Fortalezas": ["Habilidades de combate", "Diez anillos"],
        "Debilidades": ["Desconocido"]
    },
    "She-Hulk": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Impulsividad"]
    },
    "Shuri": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Inexperiencia en combate"]
    },
    "Sif": {
        "Fortalezas": ["Habilidades de combate", "Longevidad"],
        "Debilidades": ["Desconocido"]
    },
    "Sinister Strange": {
        "Fortalezas": ["Magia avanzada", "Manipulación del tiempo"],
        "Debilidades": ["Corrupción por el Darkhold"]
    },
    "Smart Hulk": {
        "Fortalezas": ["Fuerza sobrehumana", "Inteligencia"],
        "Debilidades": ["Efectos secundarios del Gauntlet"]
    },
    "Spectral Vision": {
        "Fortalezas": ["Manipulación de densidad", "Vuelo"],
        "Debilidades": ["Crisis de identidad"]
    },
    "Spider-Man": {
        "Fortalezas": ["Fuerza sobrehumana", "Sentido arácnido"],
        "Debilidades": ["Inexperiencia"]
    },
    "Star-Lord (Celestial Power)": {
        "Fortalezas": ["Poder celestial", "Vuelo"],
        "Debilidades": ["Dependencia de su conexión celestial"]
    },
    "Star-Lord": {
        "Fortalezas": ["Habilidades de combate", "Tecnología avanzada"],
        "Debilidades": ["Desconocido"]
    },
    "Stature": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Surtur": {
        "Fortalezas": ["Fuerza sobrehumana", "Piroquinesis"],
        "Debilidades": ["Vulnerabilidad cuando su corona es removida"]
    },
    "Swole Groot": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Talos The Untamed": {
        "Fortalezas": ["Cambio de forma", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Taskmaster": {
        "Fortalezas": ["Reflejos fotográficos", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Teenage Groot": {
        "Fortalezas": ["Fuerza sobrehumana", "Regeneración"],
        "Debilidades": ["Vulnerabilidad al fuego"]
    },
    "Thanos": {
        "Fortalezas": ["Fuerza sobrehumana", "Inteligencia estratégica"],
        "Debilidades": ["Arrogancia"]
    },
    "Thanos (Infinity Gauntlet)": {
        "Fortalezas": ["Fuerza sobrehumana", "Poder de las gemas del infinito"],
        "Debilidades": ["Dependencia del guantelete"]
    },
    "The Collector": {
        "Fortalezas": ["Conocimiento extenso", "Recursos infinitos"],
        "Debilidades": ["Obsesión con coleccionar"]
    },
    "The Executioner": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "The Grandmaster": {
        "Fortalezas": ["Longevidad", "Manipulación de la materia"],
        "Debilidades": ["Arrogancia"]
    },
    "The Great Protector": {
        "Fortalezas": ["Fuerza sobrehumana", "Vuelo", "Control del agua"],
        "Debilidades": ["Desconocido"]
    },
    "The Punisher": {
        "Fortalezas": ["Habilidades de combate", "Resistencia"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Thena": {
        "Fortalezas": ["Fuerza sobrehumana", "Creación de armas de energía"],
        "Debilidades": ["Desconocido"]
    },
    "Thor (Pre-Ragnarok)": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Arrogancia"]
    },
    "Thor (Awakened)": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Dependencia emocional"]
    },
    "Ultron": {
        "Fortalezas": ["Inteligencia avanzada", "Fuerza sobrehumana"],
        "Debilidades": ["Arrogancia"]
    },
    "Unworthy Thor": {
        "Fortalezas": ["Fuerza sobrehumana", "Control del rayo", "Vuelo"],
        "Debilidades": ["Baja autoestima"]
    },
    "Valkyrie": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Trauma emocional"]
    },
    "Vision": {
        "Fortalezas": ["Manipulación de densidad", "Vuelo", "Energía del rayo"],
        "Debilidades": ["Dependencia de la gema de la mente"]
    },
    "Volstagg": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Vulture": {
        "Fortalezas": ["Tecnología avanzada", "Vuelo"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Wanda Maximoff (Pre-WandaVision)": {
        "Fortalezas": ["Magia del caos", "Manipulación de la realidad"],
        "Debilidades": ["Inestabilidad emocional"]
    },
    "War Machine": {
        "Fortalezas": ["Tecnología avanzada", "Vuelo"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Warlock": {
        "Fortalezas": ["Magia avanzada", "Inmortalidad"],
        "Debilidades": ["Desconocido"]
    },
    "Wasp": {
        "Fortalezas": ["Cambio de tamaño", "Vuelo"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Whiplash": {
        "Fortalezas": ["Tecnología avanzada", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología"]
    },
    "Whiplash (Mark II)": {
        "Fortalezas": ["Tecnología avanzada", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología"]
    },
    "White Wolf": {
        "Fortalezas": ["Habilidades de combate", "Estrategia"],
        "Debilidades": ["Trauma psicológico"]
    },
    "Winter Soldier": {
        "Fortalezas": ["Fuerza sobrehumana", "Brazo biónico", "Habilidades de combate"],
        "Debilidades": ["Programación mental"]
    },
    "Wolverine": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Vulnerabilidad emocional"]
    },
    "Wong": {
        "Fortalezas": ["Magia avanzada", "Conocimiento místico"],
        "Debilidades": ["Desconocido"]
    },
    "YellowJacket": {
        "Fortalezas": ["Cambio de tamaño", "Fuerza incrementada"],
        "Debilidades": ["Dependencia de la tecnología del traje"]
    },
    "Yondu": {
        "Fortalezas": ["Control sobre su flecha", "Habilidades de combate"],
        "Debilidades": ["Humano sin poderes sobrenaturales"]
    },
    "Zeus": {
        "Fortalezas": ["Poder divino", "Inmortalidad"],
        "Debilidades": ["Arrogancia"]
    }
}

In [82]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_MCU(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_MCU:
        return pd.Series([
            strengths_weaknesses_MCU[hero_name]['Fortalezas'],
            strengths_weaknesses_MCU[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [83]:
mcu_df[['Fortalezas', 'Debilidades']] = mcu_df.apply(assign_strenghs_weaknesses_MCU, axis = 1)

In [84]:
mcu_df.info()
mcu_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  161 non-null    object
 1   nombre           176 non-null    object
 2   universo         176 non-null    object
 3   url              176 non-null    object
 4   Intelligence     176 non-null    int64 
 5   Strength         176 non-null    int64 
 6   Speed            176 non-null    object
 7   Durability       176 non-null    int64 
 8   Power            176 non-null    int64 
 9   Combat           176 non-null    int64 
 10  Tier             176 non-null    int64 
 11  Tier2            48 non-null     object
 12  superpoderes     176 non-null    object
 13  heroe            176 non-null    object
 14  Fortalezas       176 non-null    object
 15  Debilidades      176 non-null    object
dtypes: int64(6), object(10)
memory usage: 22.1+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,NaN,Agatha Harkness,Marvel Cinematic Universe,https://www.superherodb.com/agatha-harkness/10...,70,40,25,35,70,65,3,NaN,"['Durability', 'Energy Blasts', 'Energy Constr...",Agatha Harkness,"[Magia oscura, Manipulación mágica]",[Necesita absorber la magia de otros para mant...
1,Emil Blonsky,Abomination,Marvel Cinematic Universe,https://www.superherodb.com/abomination/10-12512/,65,100,25,50,65,70,3,3,"['Cold Resistance', 'Durability', 'Endurance',...",Abomination,"[Fuerza sobrehumana, Resistencia]",[Vulnerable a ataques mentales]
2,Ajak,Ajak,Marvel Cinematic Universe,https://www.superherodb.com/ajak/10-26887/,65,25,19,25,50,70,2,NaN,"['Accelerated Healing', 'Agility', 'Durability...",Ajak,"[Curación, Fuerza y durabilidad sobrehumanas]",[Vulnerable a traiciones de otros Eternos]
3,NaN,Ancient One (time stone),Marvel Cinematic Universe,https://www.superherodb.com/ancient-one-time-s...,85,8,25,15,100,100,6,6++,"['Anti-Gravity', 'Astral Projection', 'Concept...",Ancient One (Time Stone),"[Manipulación del tiempo, Magia avanzada]",[Dependencia de la Piedra del Tiempo]
4,NaN,Ancient One,Marvel Cinematic Universe,https://www.superherodb.com/ancient-one/10-12735/,85,8,25,25,90,100,4,NaN,"['Anti-Gravity', 'Astral Projection', 'Dimensi...",Ancient One,"[Magia avanzada, Longevidad]",[Vulnerabilidad física sin la Piedra del Tiempo]


DRAGON BALL

In [85]:
# Dragon ball GT
db_df = pd.read_csv('./data/Dragon_ball_GT.csv')

In [86]:
db_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nombre_completo  11 non-null     object 
 1   nombre           12 non-null     object 
 2   universo         12 non-null     object 
 3   url              12 non-null     object 
 4   Intelligence     12 non-null     int64  
 5   Strength         12 non-null     int64  
 6   Speed            12 non-null     int64  
 7   Durability       12 non-null     int64  
 8   Power            12 non-null     int64  
 9   Combat           12 non-null     int64  
 10  Tier             12 non-null     int64  
 11  Tier2            0 non-null      float64
 12  superpoderes     12 non-null     object 
 13  heroe            12 non-null     object 
dtypes: float64(1), int64(7), object(6)
memory usage: 1.4+ KB


In [87]:
strengths_weaknesses_DB = {
    "Gogeta (Super Saiyan 4)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan", "Fusión"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía", "Arrogancia"]
    },
    "Goku (Super Saiyan 3)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Técnicas de Ki"],
        "Debilidades": ["Consumo de energía", "Fatiga"]
    },
    "Goku (Super Saiyan 2)": {
        "Fortalezas": ["Fuerza mejorada", "Habilidades de combate"],
        "Debilidades": ["Consumo de energía", "Menor resistencia"]
    },
    "Goku (Super Saiyan 4)": {
        "Fortalezas": ["Fuerza y velocidad extrema", "Combate cuerpo a cuerpo", "Técnicas avanzadas de Ki"],
        "Debilidades": ["Consumo de energía", "Transformación limitada"]
    },
    "Goku Jr.": {
        "Fortalezas": ["Heredero Saiyan", "Determinación"],
        "Debilidades": ["Inexperiencia", "Fuerza limitada"]
    },
    "Goten": {
        "Fortalezas": ["Fuerza Saiyan", "Transformaciones", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Impulsividad"]
    },
    "Pan": {
        "Fortalezas": ["Heredera Saiyan", "Determinación"],
        "Debilidades": ["Inexperiencia", "Fuerza limitada"]
    },
    "Piccolo": {
        "Fortalezas": ["Fuerza y habilidades de Namekiano", "Estrategia", "Regeneración"],
        "Debilidades": ["Limitaciones físicas", "Dependencia de la meditación"]
    },
    "Son Goku": {
        "Fortalezas": ["Fuerza Saiyan", "Transformaciones", "Técnicas de Ki", "Habilidades de combate"],
        "Debilidades": ["Impulsividad", "Consumo de energía"]
    },
    "Syn Shenron": {
        "Fortalezas": ["Fuerza extrema", "Manipulación de energía"],
        "Debilidades": ["Arrogancia", "Dependencia de las esferas del dragón"]
    },
    "Trunks": {
        "Fortalezas": ["Fuerza Saiyan", "Transformaciones", "Habilidades de combate"],
        "Debilidades": ["Impulsividad", "Dependencia emocional"]
    },
    "Uub": {
        "Fortalezas": ["Potencial humano", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Confianza en sí mismo"]
    }
}


In [88]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_DB_GT(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_DB:
        return pd.Series([
            strengths_weaknesses_DB[hero_name]['Fortalezas'],
            strengths_weaknesses_DB[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [89]:
db_df[['Fortalezas', 'Debilidades']] = db_df.apply(assign_strenghs_weaknesses_DB_GT, axis = 1)

In [90]:
db_df.info()
db_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nombre_completo  11 non-null     object 
 1   nombre           12 non-null     object 
 2   universo         12 non-null     object 
 3   url              12 non-null     object 
 4   Intelligence     12 non-null     int64  
 5   Strength         12 non-null     int64  
 6   Speed            12 non-null     int64  
 7   Durability       12 non-null     int64  
 8   Power            12 non-null     int64  
 9   Combat           12 non-null     int64  
 10  Tier             12 non-null     int64  
 11  Tier2            0 non-null      float64
 12  superpoderes     12 non-null     object 
 13  heroe            12 non-null     object 
 14  Fortalezas       12 non-null     object 
 15  Debilidades      12 non-null     object 
dtypes: float64(1), int64(7), object(8)
memory usage: 1.6+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,Gogeta,Gogeta (Super Saiyan 4),Dragon Ball GT,https://www.superherodb.com/gogeta-super-saiya...,80,100,100,100,100,100,6,NaN,"['Accelerated Development', 'Accelerated Heali...",Gogeta (Super Saiyan 4),"[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía, Arrogancia]"
1,Kakarot,Goku (Super Saiyan 3),Dragon Ball GT,https://www.superherodb.com/goku-super-saiyan-...,55,100,100,85,85,100,5,NaN,"['Accelerated Development', 'Acrobatics', 'Aft...",Goku (Super Saiyan 3),"[Fuerza sobrehumana, Habilidades de combate, T...","[Consumo de energía, Fatiga]"
2,Kakarot,Goku (Super Saiyan 2),Dragon Ball GT,https://www.superherodb.com/goku-super-saiyan-...,55,100,100,85,80,100,5,NaN,"['Accelerated Development', 'Acrobatics', 'Aft...",Goku (Super Saiyan 2),"[Fuerza mejorada, Habilidades de combate]","[Consumo de energía, Menor resistencia]"
3,Kakarot,Goku (Super Saiyan 4),Dragon Ball GT,https://www.superherodb.com/goku-super-saiyan-...,55,100,100,95,100,100,6,NaN,"['Accelerated Development', 'Accelerated Heali...",Goku (Super Saiyan 4),"[Fuerza y velocidad extrema, Combate cuerpo a ...","[Consumo de energía, Transformación limitada]"
4,NaN,Goku Jr.,Dragon Ball GT,https://www.superherodb.com/goku-jr/10-64674/,65,100,75,75,70,50,4,NaN,"['Acrobatics', 'Chi Manipulation', 'Danmaku', ...",Goku Jr.,"[Heredero Saiyan, Determinación]","[Inexperiencia, Fuerza limitada]"


In [91]:
# Dragon ball verse

db_verse_df = pd.read_csv('./data/Dragon_ball_verse.csv')

In [92]:
# Creo nuevo data, solo con nombre
data_DBverse_nombre = db_verse_df[['nombre']]
data_DBverse_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  89 non-null     object
dtypes: object(1)
memory usage: 844.0+ bytes


In [93]:
# Guardar data en .csv
data_DBverse_nombre.to_csv('./data/DBverse_nombres.csv', index = False)

In [94]:
data_DBverse_nombre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  89 non-null     object
dtypes: object(1)
memory usage: 844.0+ bytes


In [95]:
strengths_weaknesses_DBverse = {
    "Android 16": {
        "Fortalezas": ["Fuerza sobrehumana", "Inmortalidad"],
        "Debilidades": ["Emocionalidad", "Dependencia de órdenes"]
    },
    "Android 17": {
        "Fortalezas": ["Fuerza sobrehumana", "Inmortalidad"],
        "Debilidades": ["Arrogancia", "Independencia"]
    },
    "Android 18": {
        "Fortalezas": ["Fuerza sobrehumana", "Inmortalidad"],
        "Debilidades": ["Arrogancia", "Independencia"]
    },
    "Android 19": {
        "Fortalezas": ["Fuerza sobrehumana", "Absorción de energía"],
        "Debilidades": ["Dependencia de energía", "Desconocimiento de emociones"]
    },
    "Aniraza": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Consumo de energía", "Falta de estrategia"]
    },
    "Arak": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Bandages The Mummy": {
        "Fortalezas": ["Resistencia", "Fuerza física"],
        "Debilidades": ["Velocidad limitada", "Dependencia de órdenes"]
    },
    "Bardock (Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Impulsividad"]
    },
    "Bardock": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Impulsividad"]
    },
    "Basil": {
        "Fortalezas": ["Fuerza sobrehumana", "Velocidad"],
        "Debilidades": ["Arrogancia", "Inexperiencia"]
    },
    "Beerus": {
        "Fortalezas": ["Poder divino", "Fuerza sobrehumana", "Vuelo"],
        "Debilidades": ["Arrogancia", "Dependencia emocional"]
    },
    "Belmod": {
        "Fortalezas": ["Poder divino", "Fuerza sobrehumana"],
        "Debilidades": ["Desconocido"]
    },
    "Bergamo": {
        "Fortalezas": ["Fuerza sobrehumana", "Absorción de energía"],
        "Debilidades": ["Consumo de energía", "Dependencia de ataques"]
    },
    "Biarra": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Desconocido"]
    },
    "Bido": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Bikal": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Black Frieza": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones"],
        "Debilidades": ["Arrogancia", "Malicia"]
    },
    "Bojack": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Impulsividad"]
    },
    "Borareta": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Desconocido"]
    },
    "Botamo": {
        "Fortalezas": ["Fuerza sobrehumana", "Absorción de energía"],
        "Debilidades": ["Lentitud", "Dependencia de ataques"]
    },
    "Broly (Z)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Falta de control"]
    },
    "Broly (Ikari)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Falta de control"]
    },
    "Broly (Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Falta de control"]
    },
    "Broly": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Falta de control"]
    },
    "Bujin": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Burter": {
        "Fortalezas": ["Velocidad extrema", "Fuerza sobrehumana"],
        "Debilidades": ["Arrogancia", "Dependencia de velocidad"]
    },
    "Buu (Good)": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Inocencia", "Falta de estrategia"]
    },
    "Buu (Grand Supreme Kai)": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana"],
        "Debilidades": ["Desconocido"]
    },
    "Buuhan": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana", "Absorción de energía"],
        "Debilidades": ["Arrogancia", "Dependencia de absorciones"]
    },
    "Buutenks": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana", "Absorción de energía"],
        "Debilidades": ["Arrogancia", "Dependencia de absorciones"]
    },
    "Cabba (Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones Saiyan"],
        "Debilidades": ["Inexperiencia", "Dependencia emocional"]
    },
    "Cabba (Super Saiyan 2)": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones Saiyan"],
        "Debilidades": ["Inexperiencia", "Dependencia emocional"]
    },
    "Cabba": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Dependencia emocional"]
    },
    "Dende": {
        "Fortalezas": ["Curación", "Sabiduría"],
        "Debilidades": ["Fuerza limitada", "Dependencia de protección"]
    },
    "Dercori": {
        "Fortalezas": ["Desconocido"],
        "Debilidades": ["Desconocido"]
    },
    "Dodoria": {
        "Fortalezas": ["Fuerza sobrehumana", "Resistencia"],
        "Debilidades": ["Arrogancia", "Dependencia de órdenes"]
    },
    "Dore": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Desconocido"]
    },
    "Dr Gero": {
        "Fortalezas": ["Inteligencia", "Tecnología avanzada"],
        "Debilidades": ["Fuerza limitada", "Dependencia de androides"]
    },
    "Dyspo (Super Maximum Light Speed)": {
        "Fortalezas": ["Velocidad extrema", "Fuerza sobrehumana"],
        "Debilidades": ["Dependencia de velocidad", "Consumo de energía"]
    },
    "Dyspo": {
        "Fortalezas": ["Velocidad extrema", "Fuerza sobrehumana"],
        "Debilidades": ["Dependencia de velocidad", "Consumo de energía"]
    },
    "Frieza": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones", "Inteligencia"],
        "Debilidades": ["Arrogancia", "Crueldad"]
    },
    "Frieza (First Form)": {
        "Fortalezas": ["Fuerza sobrehumana", "Inteligencia"],
        "Debilidades": ["Arrogancia", "Crueldad"]
    },
    "Frieza (Second Form)": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones"],
        "Debilidades": ["Arrogancia", "Crueldad"]
    },
    "Frieza (Third Form)": {
        "Fortalezas": ["Fuerza sobrehumana", "Transformaciones"],
        "Debilidades": ["Arrogancia", "Crueldad"]
    },
    "Fusion Zamasu": {
        "Fortalezas": ["Fuerza divina", "Inmortalidad"],
        "Debilidades": ["Arrogancia", "Instabilidad mental"]
    },
    "Future Trunks (Cell Saga)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Impulsividad", "Emocionalidad"]
    },
    "Gogeta": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    },
    "Gogeta (Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    },
    "Gogeta (Super Saiyan Blue)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    },
    "Gohan (Beast)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Inestabilidad emocional", "Impulsividad"]
    },
    "Gohan (Potential Unleashed)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Transformaciones"],
        "Debilidades": ["Inexperiencia", "Dependencia emocional"]
    },
    "Gohan (SS2)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Inexperiencia", "Impulsividad"]
    },
    "Gohan (Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Inexperiencia", "Impulsividad"]
    },
    "Gohan": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Dependencia emocional"]
    },
    "Goku (Super Saiyan Blue Kaioken)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Fatiga"]
    },
    "Goku": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Transformaciones Saiyan", "Técnicas de Ki"],
        "Debilidades": ["Impulsividad", "Consumo de energía"]
    },
    "Goku (Mastered Ultra Instinct)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Reflejos perfectos", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Dependencia emocional"]
    },
    "Goku (Saiyan Saga)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Impulsividad", "Inexperiencia"]
    },
    "Goku (True Ultra Instinct)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Reflejos perfectos", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Dependencia emocional"]
    },
    "Goku (Super Saiyan God)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Velocidad", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Dependencia emocional"]
    },
    "Goku (Super Saiyan 2)": {
        "Fortalezas": ["Fuerza mejorada", "Habilidades de combate"],
        "Debilidades": ["Consumo de energía", "Menor resistencia"]
    },
    "Goku (Ultra Instinct)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Reflejos perfectos"],
        "Debilidades": ["Consumo de energía", "Dependencia emocional"]
    },
    "Goku (Super Saiyan Blue)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Fatiga"]
    },
    "Goku (Super Saiyan 3)": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate", "Técnicas de Ki"],
        "Debilidades": ["Consumo de energía", "Fatiga"]
    },
    "Goku Black (Super Saiyan Rosé)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Malicia"]
    },
    "Goku Black": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Malicia"]
    },
    "Goten (Super Saiyan)": {
        "Fortalezas": ["Fuerza Saiyan", "Transformaciones", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Impulsividad"]
    },
    "Goten": {
        "Fortalezas": ["Fuerza Saiyan", "Transformaciones", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Impulsividad"]
    },
    "Gotenks (Super Saiyan 3)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    },
    "Gotenks (SS2)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    },
    "Grand Priest": {
        "Fortalezas": ["Poder divino extremo", "Inteligencia"],
        "Debilidades": ["Desconocido"]
    },
    "Jaco": {
        "Fortalezas": ["Velocidad", "Habilidades de combate"],
        "Debilidades": ["Fuerza limitada", "Arrogancia"]
    },
    "Kami": {
        "Fortalezas": ["Sabiduría", "Habilidades de Namekiano"],
        "Debilidades": ["Fuerza limitada", "Dependencia de Piccolo"]
    },
    "Kid Buu": {
        "Fortalezas": ["Regeneración", "Fuerza sobrehumana", "Destrucción"],
        "Debilidades": ["Falta de control", "Impulsividad"]
    },
    "Kid Gohan": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Dependencia emocional"]
    },
    "Kid Goku": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Inexperiencia", "Impulsividad"]
    },
    "King Vegeta": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Crueldad"]
    },
    "Krillin": {
        "Fortalezas": ["Habilidades de combate", "Estrategia"],
        "Debilidades": ["Fuerza limitada", "Dependencia emocional"]
    },
    "Vegeta (Ascended Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Consumo de energía"]
    },
    "Vegeta (Unbound Ultra Ego)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Arrogancia", "Consumo de energía"]
    },
    "Vegeta (Super Saiyan Blue Evolved)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Arrogancia"]
    },
    "Vegeta (Super Saiyan 2)": {
        "Fortalezas": ["Fuerza mejorada", "Habilidades de combate"],
        "Debilidades": ["Consumo de energía", "Menor resistencia"]
    },
    "Vegeta (Super Saiyan Blue)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Fatiga"]
    },
    "Vegeta (Hakai Energy)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Arrogancia"]
    },
    "Vegeta (Super Saiyan)": {
        "Fortalezas": ["Fuerza mejorada", "Habilidades de combate"],
        "Debilidades": ["Consumo de energía", "Menor resistencia"]
    },
    "Vegeta": {
        "Fortalezas": ["Fuerza sobrehumana", "Habilidades de combate"],
        "Debilidades": ["Arrogancia", "Impulsividad"]
    },
    "Vegeta (Super Saiyan God)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Velocidad", "Transformaciones Saiyan"],
        "Debilidades": ["Consumo de energía", "Dependencia emocional"]
    },
    "Vegito": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    },
    "Vegito (Super Saiyan)": {
        "Fortalezas": ["Fuerza sobrehumana extrema", "Habilidades de combate", "Transformaciones Saiyan"],
        "Debilidades": ["Tiempo limitado", "Consumo de energía"]
    }
}


In [96]:
# Asignar fortalezas y debilidades
def assign_strenghs_weaknesses_DB_verse(row):
    hero_name = row['nombre']
    if hero_name in strengths_weaknesses_DBverse:
        return pd.Series([
            strengths_weaknesses_DBverse[hero_name]['Fortalezas'],
            strengths_weaknesses_DBverse[hero_name]['Debilidades']
        ])
    else:
        return pd.Series([['Desconocido'], ['Desconocido']])

In [97]:
db_verse_df[['Fortalezas', 'Debilidades']] = db_verse_df.apply(assign_strenghs_weaknesses_DB_verse, axis = 1)

In [98]:
db_verse_df.info()
db_verse_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  74 non-null     object
 1   nombre           89 non-null     object
 2   universo         89 non-null     object
 3   url              89 non-null     object
 4   Intelligence     89 non-null     int64 
 5   Strength         89 non-null     int64 
 6   Speed            89 non-null     int64 
 7   Durability       89 non-null     int64 
 8   Power            89 non-null     int64 
 9   Combat           89 non-null     int64 
 10  Tier             89 non-null     int64 
 11  Tier2            6 non-null      object
 12  superpoderes     89 non-null     object
 13  heroe            89 non-null     object
 14  Fortalezas       89 non-null     object
 15  Debilidades      89 non-null     object
dtypes: int64(7), object(9)
memory usage: 11.3+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,NaN,Android 16,Dragon Ball Verse,https://www.superherodb.com/android-16/10-1238/,90,100,80,75,55,100,5,NaN,"['Accelerated Development', 'Acrobatics', 'Agi...",Android 16,"[Fuerza sobrehumana, Inmortalidad]","[Emocionalidad, Dependencia de órdenes]"
1,Lapis,Android 17,Dragon Ball Verse,https://www.superherodb.com/android-17/10-1217/,85,100,100,100,100,100,6,NaN,"['Acrobatics', 'Afterimage Creation', 'Agility...",Android 17,"[Fuerza sobrehumana, Inmortalidad]","[Arrogancia, Independencia]"
2,Lazuli,Android 18,Dragon Ball Verse,https://www.superherodb.com/android-18/10-1218/,60,100,100,90,90,100,5,NaN,"['Acrobatics', 'Agility', 'Cyborgization', 'Da...",Android 18,"[Fuerza sobrehumana, Inmortalidad]","[Arrogancia, Independencia]"
3,NaN,Android 19,Dragon Ball Verse,https://www.superherodb.com/android-19/10-1239/,80,100,75,60,45,90,4,NaN,"['Acrobatics', 'Agility', 'Bionics', 'Cold Res...",Android 19,"[Fuerza sobrehumana, Absorción de energía]","[Dependencia de energía, Desconocimiento de em..."
4,Aniraza,Aniraza,Dragon Ball Verse,https://www.superherodb.com/aniraza/10-13451/,80,100,100,100,100,100,7,7+,"['Ability Shift', 'Accelerated Development', '...",Aniraza,"[Fuerza sobrehumana, Habilidades de combate]","[Consumo de energía, Falta de estrategia]"


In [99]:
all_data = pd.concat([dc_df, theboys_df, mcu_df, db_df, db_verse_df])

In [100]:
all_data.info()
all_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 88
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  324 non-null    object
 1   nombre           385 non-null    object
 2   universo         385 non-null    object
 3   url              385 non-null    object
 4   Intelligence     385 non-null    int64 
 5   Strength         385 non-null    int64 
 6   Speed            385 non-null    object
 7   Durability       385 non-null    int64 
 8   Power            385 non-null    int64 
 9   Combat           385 non-null    int64 
 10  Tier             385 non-null    int64 
 11  Tier2            85 non-null     object
 12  superpoderes     385 non-null    object
 13  heroe            385 non-null    object
 14  Fortalezas       385 non-null    object
 15  Debilidades      385 non-null    object
dtypes: int64(6), object(10)
memory usage: 51.1+ KB


,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,superpoderes,heroe,Fortalezas,Debilidades
0,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,"['Animal Control', 'Durability', 'Stamina', 'S...",Aquaman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...
1,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,"['Accelerated Healing', 'Chain Manipulation', ...",Ares,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
2,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,"['Jaw Strength', 'Large Size', 'Size Changing'...",Atom Smasher,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,"['Bloodlust', 'Dexterity', 'Disguise Mastery',...",Batman,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,"['Afterimage Creation', 'Agility', 'Cold Resis...",Black Adam,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"


In [101]:
# Guardar final_data
all_data.to_csv('./data/Total_data_heores.csv', index = False)

#### ENFRENTAMIENTOS

In [102]:
name_data = all_data[['nombre']]
name_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 88
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  385 non-null    object
dtypes: object(1)
memory usage: 6.0+ KB


In [103]:
heroes = name_data['nombre'].tolist()
#heroes

In [104]:
# Generar combinaciones ficticias de enfrentamientos
battle = list(itertools.combinations(heroes, 2))
#battle

In [105]:
# Convertir a dataframe
battle_df = pd.DataFrame(battle, columns = ['Heroe_1', 'Heroe_2'])

In [106]:
battle_df.info()
battle_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73920 entries, 0 to 73919
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Heroe_1  73920 non-null  object
 1   Heroe_2  73920 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


,Heroe_1,Heroe_2
0,Aquaman,Ares
1,Aquaman,Atom Smasher
2,Aquaman,Batman
3,Aquaman,Black Adam
4,Aquaman,Black Canary


In [107]:
df_strengh_weakness = all_data[['nombre', 'Fortalezas', 'Debilidades']]
df_strengh_weakness.head()

,nombre,Fortalezas,Debilidades
0,Aquaman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...
1,Ares,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
2,Atom Smasher,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Batman,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Black Adam,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"


In [108]:
df_strengh_weakness.to_csv('./data/fortalezas_debilidades.csv', index = False)

In [109]:
# Guardar enfrentamientos
battle_df.to_csv('./data/Battle_data.csv', index = False)

In [110]:
# Merge the two dataframes based on hero names
merged_data_1 = pd.merge(battle_df, df_strengh_weakness, left_on='Heroe_1', right_on='nombre')
merged_data_complete = pd.merge(merged_data_1, df_strengh_weakness, left_on='Heroe_2', right_on='nombre', suffixes=('_1', '_2'))

# Select and rename relevant columns
final_data = merged_data_complete[['Heroe_1', 'Heroe_2', 'Fortalezas_1', 'Debilidades_1', 'Fortalezas_2', 'Debilidades_2']]
final_data.columns = ['Heroe 1', 'Heroe 2', 'Fortaleza 1', 'Debilidad 1', 'Fortaleza 2', 'Debilidad 2']


In [111]:
final_data.info()
final_data.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79387 entries, 0 to 79386
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Heroe 1      79387 non-null  object
 1   Heroe 2      79387 non-null  object
 2   Fortaleza 1  79387 non-null  object
 3   Debilidad 1  79387 non-null  object
 4   Fortaleza 2  79387 non-null  object
 5   Debilidad 2  79387 non-null  object
dtypes: object(6)
memory usage: 3.6+ MB


,Heroe 1,Heroe 2,Fortaleza 1,Debilidad 1,Fortaleza 2,Debilidad 2
0,Aquaman,Ares,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
1,Aquaman,Atom Smasher,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
2,Ares,Atom Smasher,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Aquaman,Batman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Ares,Batman,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
5,Atom Smasher,Batman,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...","[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
6,Aquaman,Black Adam,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"
7,Ares,Black Adam,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"
8,Atom Smasher,Black Adam,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...","[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"
9,Batman,Black Adam,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...","[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"


In [112]:
# Update the function to return 1, 2, or 0 based on the winner
def determine_winner_numeric(row):
    # Count the number of strengths and weaknesses
    strengths_1 = len(row['Fortaleza 1'])
    weaknesses_1 = len(row['Debilidad 1'])
    strengths_2 = len(row['Fortaleza 2'])
    weaknesses_2 = len(row['Debilidad 2'])
    
    # Determine advantage based on strengths against weaknesses
    advantage_1 = sum(1 for strength in row['Fortaleza 1'] if strength in row['Debilidad 2'])
    advantage_2 = sum(1 for strength in row['Fortaleza 2'] if strength in row['Debilidad 1'])
    
    # Determine the winner
    if advantage_1 > advantage_2:
        return 1
    elif advantage_2 > advantage_1:
        return 2
    else:
        # If advantages are equal, compare the number of strengths
        if strengths_1 > strengths_2:
            return 1
        elif strengths_2 > strengths_1:
            return 2
        else:
            return 0

# Apply the function to determine winners
final_data['Ganador'] = final_data.apply(determine_winner_numeric, axis=1)

In [113]:
final_data.to_csv('./data/Batallas_resultados.csv')

In [125]:
final_data.info()
final_data.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79387 entries, 0 to 79386
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Heroe 1      79387 non-null  object
 1   Heroe 2      79387 non-null  object
 2   Fortaleza 1  79387 non-null  object
 3   Debilidad 1  79387 non-null  object
 4   Fortaleza 2  79387 non-null  object
 5   Debilidad 2  79387 non-null  object
 6   Ganador      79387 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 4.2+ MB


,Heroe 1,Heroe 2,Fortaleza 1,Debilidad 1,Fortaleza 2,Debilidad 2,Ganador
0,Aquaman,Ares,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,1
1,Aquaman,Atom Smasher,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...",1
2,Ares,Atom Smasher,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...",1
3,Aquaman,Batman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...",0
4,Ares,Batman,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...",2
5,Atom Smasher,Batman,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...","[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...",2
6,Aquaman,Black Adam,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]",0
7,Ares,Black Adam,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]",2
8,Atom Smasher,Black Adam,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...","[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]",2
9,Batman,Black Adam,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...","[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]",0


DATA FINAL CON RESULTADOS 

In [115]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 385 entries, 0 to 88
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre_completo  324 non-null    object
 1   nombre           385 non-null    object
 2   universo         385 non-null    object
 3   url              385 non-null    object
 4   Intelligence     385 non-null    int64 
 5   Strength         385 non-null    int64 
 6   Speed            385 non-null    object
 7   Durability       385 non-null    int64 
 8   Power            385 non-null    int64 
 9   Combat           385 non-null    int64 
 10  Tier             385 non-null    int64 
 11  Tier2            85 non-null     object
 12  superpoderes     385 non-null    object
 13  heroe            385 non-null    object
 14  Fortalezas       385 non-null    object
 15  Debilidades      385 non-null    object
dtypes: int64(6), object(10)
memory usage: 51.1+ KB


In [116]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79387 entries, 0 to 79386
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Heroe 1      79387 non-null  object
 1   Heroe 2      79387 non-null  object
 2   Fortaleza 1  79387 non-null  object
 3   Debilidad 1  79387 non-null  object
 4   Fortaleza 2  79387 non-null  object
 5   Debilidad 2  79387 non-null  object
 6   Ganador      79387 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 4.2+ MB


In [117]:
all_data_renamed = all_data.rename(columns={
    'nombre': 'Heroe 1', 
    'universo': 'Universo 1',
    'Intelligence': 'Intelligence 1',
    'Strength': 'Strength 1',
    'Speed': 'Speed 1',
    'Durability': 'Durability 1',
    'Power': 'Power 1',
    'Combat': 'Combat 1',
    'superpoderes': 'Superpoderes 1'
})


In [118]:
all_data_renamed

,nombre_completo,Heroe 1,Universo 1,url,Intelligence 1,Strength 1,Speed 1,Durability 1,Power 1,Combat 1,Tier,Tier2,Superpoderes 1,heroe,Fortalezas,Debilidades
0,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,"['Animal Control', 'Durability', 'Stamina', 'S...",Aquaman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...
1,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,"['Accelerated Healing', 'Chain Manipulation', ...",Ares,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
2,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,"['Jaw Strength', 'Large Size', 'Size Changing'...",Atom Smasher,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,"['Bloodlust', 'Dexterity', 'Disguise Mastery',...",Batman,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,"['Afterimage Creation', 'Agility', 'Cold Resis...",Black Adam,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Vegeta IV,Vegeta (Super Saiyan),Dragon Ball Verse,https://www.superherodb.com/vegeta-super-saiya...,70,100,100,100,100,100,6,NaN,"['Acrobatics', 'Adaptation', 'Afterimage Creat...",Vegeta (Super Saiyan),"[Fuerza mejorada, Habilidades de combate]","[Consumo de energía, Menor resistencia]"
85,Vegeta IV,Vegeta,Dragon Ball Verse,https://www.superherodb.com/vegeta/10-1206/,65,100,100,100,100,100,6,NaN,"['Acrobatics', 'Adaptation', 'Afterimage Creat...",Vegeta,"[Fuerza sobrehumana, Habilidades de combate]","[Arrogancia, Impulsividad]"
86,Vegeta IV,Vegeta (Super Saiyan God),Dragon Ball Verse,https://www.superherodb.com/vegeta-super-saiya...,60,100,90,70,80,100,7,NaN,"['Accelerated Healing', 'Agility', 'Aura', 'Ch...",Vegeta (Super Saiyan God),"[Fuerza sobrehumana extrema, Velocidad, Transf...","[Consumo de energía, Dependencia emocional]"
87,Vegetto,Vegito,Dragon Ball Verse,https://www.superherodb.com/vegito/10-11729/,60,100,100,100,100,100,6,NaN,"['Acid Resistants', 'Acrobatics', 'Afterimage ...",Vegito,"[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía]"


In [119]:
all_data_renamed_2 = all_data.rename(columns={
    'nombre': 'Heroe 2', 
    'universo': 'Universo 2',
    'Intelligence': 'Intelligence 2',
    'Strength': 'Strength 2',
    'Speed': 'Speed 2',
    'Durability': 'Durability 2',
    'Power': 'Power 2',
    'Combat': 'Combat 2',
    'superpoderes': 'Superpoderes 2'
})

In [120]:
all_data_renamed_2

,nombre_completo,Heroe 2,Universo 2,url,Intelligence 2,Strength 2,Speed 2,Durability 2,Power 2,Combat 2,Tier,Tier2,Superpoderes 2,heroe,Fortalezas,Debilidades
0,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,"['Animal Control', 'Durability', 'Stamina', 'S...",Aquaman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...
1,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,"['Accelerated Healing', 'Chain Manipulation', ...",Ares,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...
2,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,"['Jaw Strength', 'Large Size', 'Size Changing'...",Atom Smasher,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con..."
3,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,"['Bloodlust', 'Dexterity', 'Disguise Mastery',...",Batman,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil..."
4,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,"['Afterimage Creation', 'Agility', 'Cold Resis...",Black Adam,"[Fuerza sobrehumana, Velocidad, Magia, Vuelo, ...","[Vulnerable a la magia poderosa, Arrogancia]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Vegeta IV,Vegeta (Super Saiyan),Dragon Ball Verse,https://www.superherodb.com/vegeta-super-saiya...,70,100,100,100,100,100,6,NaN,"['Acrobatics', 'Adaptation', 'Afterimage Creat...",Vegeta (Super Saiyan),"[Fuerza mejorada, Habilidades de combate]","[Consumo de energía, Menor resistencia]"
85,Vegeta IV,Vegeta,Dragon Ball Verse,https://www.superherodb.com/vegeta/10-1206/,65,100,100,100,100,100,6,NaN,"['Acrobatics', 'Adaptation', 'Afterimage Creat...",Vegeta,"[Fuerza sobrehumana, Habilidades de combate]","[Arrogancia, Impulsividad]"
86,Vegeta IV,Vegeta (Super Saiyan God),Dragon Ball Verse,https://www.superherodb.com/vegeta-super-saiya...,60,100,90,70,80,100,7,NaN,"['Accelerated Healing', 'Agility', 'Aura', 'Ch...",Vegeta (Super Saiyan God),"[Fuerza sobrehumana extrema, Velocidad, Transf...","[Consumo de energía, Dependencia emocional]"
87,Vegetto,Vegito,Dragon Ball Verse,https://www.superherodb.com/vegito/10-11729/,60,100,100,100,100,100,6,NaN,"['Acid Resistants', 'Acrobatics', 'Afterimage ...",Vegito,"[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía]"


In [121]:
final_data_merged = final_data.merge(all_data_renamed[['Heroe 1', 'Universo 1', 'Intelligence 1', 'Strength 1', 'Speed 1', 'Durability 1', 'Power 1', 'Combat 1', 'Superpoderes 1']],
                                     on='Heroe 1', how='left')

final_data_merged = final_data_merged.merge(all_data_renamed_2[['Heroe 2', 'Universo 2', 'Intelligence 2', 'Strength 2', 'Speed 2', 'Durability 2', 'Power 2', 'Combat 2', 'Superpoderes 2']],
                                            on='Heroe 2', how='left')
final_data_merged

,Heroe 1,Heroe 2,Fortaleza 1,Debilidad 1,Fortaleza 2,Debilidad 2,Ganador,Universo 1,Intelligence 1,Strength 1,...,Combat 1,Superpoderes 1,Universo 2,Intelligence 2,Strength 2,Speed 2,Durability 2,Power 2,Combat 2,Superpoderes 2
0,Aquaman,Ares,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,1,DC Extended Universe,65,50,...,80,"['Animal Control', 'Durability', 'Stamina', 'S...",DC Extended Universe,90,100,50,50,70,100,"['Accelerated Healing', 'Chain Manipulation', ..."
1,Aquaman,Atom Smasher,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...",1,DC Extended Universe,65,50,...,80,"['Animal Control', 'Durability', 'Stamina', 'S...",DC Extended Universe,60,30,25,25,40,20,"['Jaw Strength', 'Large Size', 'Size Changing'..."
2,Ares,Atom Smasher,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Cambio de tamaño, Fuerza incrementada con el ...","[Difícil de controlar su tamaño, Problemas con...",1,DC Extended Universe,90,100,...,100,"['Accelerated Healing', 'Chain Manipulation', ...",DC Extended Universe,60,30,25,25,40,20,"['Jaw Strength', 'Large Size', 'Size Changing'..."
3,Aquaman,Batman,"[Control sobre animales marinos, Fuerza sobreh...",[Necesidad de estar en contacto con el agua re...,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...",0,DC Extended Universe,65,50,...,80,"['Animal Control', 'Durability', 'Stamina', 'S...",DC Extended Universe,80,13,11,15,20,95,"['Bloodlust', 'Dexterity', 'Disguise Mastery',..."
4,Ares,Batman,"[Fuerza y durabilidad sobrehumana, Habilidades...",[Dependencia del caos y conflicto para mantene...,"[Inteligencia de nivel genio, Maestro en artes...","[Humano sin poderes sobrenaturales, Vulnerabil...",2,DC Extended Universe,90,100,...,100,"['Accelerated Healing', 'Chain Manipulation', ...",DC Extended Universe,80,13,11,15,20,95,"['Bloodlust', 'Dexterity', 'Disguise Mastery',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90862,Vegeta (Hakai Energy),Vegito (Super Saiyan),"[Fuerza sobrehumana extrema, Habilidades de co...","[Consumo de energía, Arrogancia]","[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía]",0,Dragon Ball Verse,85,100,...,100,"['Accelerated Development', 'Acrobatics', 'Ada...",Dragon Ball Verse,60,100,100,100,100,100,"['Acrobatics', 'Afterimage Creation', 'Agility..."
90863,Vegeta (Super Saiyan),Vegito (Super Saiyan),"[Fuerza mejorada, Habilidades de combate]","[Consumo de energía, Menor resistencia]","[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía]",2,Dragon Ball Verse,70,100,...,100,"['Acrobatics', 'Adaptation', 'Afterimage Creat...",Dragon Ball Verse,60,100,100,100,100,100,"['Acrobatics', 'Afterimage Creation', 'Agility..."
90864,Vegeta,Vegito (Super Saiyan),"[Fuerza sobrehumana, Habilidades de combate]","[Arrogancia, Impulsividad]","[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía]",2,Dragon Ball Verse,65,100,...,100,"['Acrobatics', 'Adaptation', 'Afterimage Creat...",Dragon Ball Verse,60,100,100,100,100,100,"['Acrobatics', 'Afterimage Creation', 'Agility..."
90865,Vegeta (Super Saiyan God),Vegito (Super Saiyan),"[Fuerza sobrehumana extrema, Velocidad, Transf...","[Consumo de energía, Dependencia emocional]","[Fuerza sobrehumana extrema, Habilidades de co...","[Tiempo limitado, Consumo de energía]",0,Dragon Ball Verse,60,100,...,100,"['Accelerated Healing', 'Agility', 'Aura', 'Ch...",Dragon Ball Verse,60,100,100,100,100,100,"['Acrobatics', 'Afterimage Creation', 'Agility..."


In [122]:
ordered_columns = ['Heroe 1', 'Heroe 2', 
                   'Fortaleza 1', 'Fortaleza 2', 
                   'Debilidad 1', 'Debilidad 2',
                   'Universo 1', 'Universo 2',
                   'Intelligence 1', 'Strength 1', 'Speed 1', 'Durability 1', 'Power 1', 'Combat 1', 
                   'Intelligence 2', 'Strength 2', 'Speed 2', 'Durability 2', 'Power 2', 'Combat 2', 
                   'Superpoderes 1', 'Superpoderes 2',
                   'Ganador'
]

In [123]:
final_data_merged = final_data_merged[ordered_columns]

In [124]:
#final_data_merged.to_csv('./data/Heroes_final_data.csv', index = False)